<a href="https://colab.research.google.com/github/Basha15/Amdox-Internship/blob/main/Employee_Wellness_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

print("⏳ Installing AI & Visualization libraries... (This may take a minute)")
try:
    import transformers
    import plotly
except ImportError:
    install("transformers")
    install("plotly")
    install("pandas")
    print("✅ Libraries installed successfully!")

# --- STEP 2: IMPORTS ---
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from transformers import pipeline
from datetime import datetime, timedelta
import random
import warnings

warnings.filterwarnings('ignore')

# --- STEP 3: DATA SIMULATION MODULE ---
def generate_employee_data(num_days=30, num_employees=5):
    """
    Simulates employee text logs (e.g., Daily Standup updates, Slack messages)
    Refers to PDF Feature 1: Real-Time Emotion Detection inputs.
    """
    print(f"\n📝 Simulating {num_days} days of communication logs for {num_employees} employees...")

    employees = ['Alice', 'Bob', 'Charlie', 'Diana', 'Ethan']

    # Templates for different emotional states
    positive_texts = [
        "Great progress today, finished the module early!",
        "Excited to start the new feature, team is syncing well.",
        "Solved that complex bug, feeling very productive.",
        "Client meeting went great, approved the design.",
        "Learning a lot from this project, enjoying the challenge."
    ]

    stressed_texts = [
        "Overwhelmed with the deadline approaching.",
        "Too many meetings, can't focus on coding.",
        "Struggling to fix this bug, it's very frustration.",
        "Need help with the backend, I'm stuck.",
        "Working late again to catch up."
    ]

    neutral_texts = [
        "Completed the documentation.",
        "Attended the sync call.",
        "Updating the jira tickets.",
        "Sent the report to the manager.",
        "Routine maintenance tasks done."
    ]

    data = []
    base_date = datetime.now() - timedelta(days=num_days)

    for i in range(num_days):
        current_date = base_date + timedelta(days=i)

        for emp in employees:
            # Simulate mood swings (Random but with some consistency)
            rand_val = random.random()
            if rand_val < 0.2: # 20% chance of stress
                text = random.choice(stressed_texts)
            elif rand_val < 0.6: # 40% chance of positive
                text = random.choice(positive_texts)
            else:
                text = random.choice(neutral_texts)

            data.append({
                'Date': current_date,
                'Employee': emp,
                'Log_Text': text
            })

    return pd.DataFrame(data)

# --- STEP 4: AI EMOTION ANALYSIS MODEL ---
def analyze_emotions(df):
    """
    Uses a pre-trained NLP model to classify text into emotions.
    Refers to PDF Feature 1 & 3: Emotion Detection & Historical Tracking.
    """
    print("🧠 Loading NLP Emotion Model (DistilRoBERTa)...")

    # Using a model fine-tuned for emotion detection
    emotion_classifier = pipeline("text-classification",
                                model="j-hartmann/emotion-english-distilroberta-base",
                                return_all_scores=False)

    print("⚡ Analyzing employee logs...")

    # Batch processing for speed
    texts = df['Log_Text'].tolist()
    results = emotion_classifier(texts)

    df['Emotion'] = [r['label'] for r in results]
    df['Confidence'] = [r['score'] for r in results]

    # Map emotions to "Mood Score" for numerical analysis (1-10)
    # Joy/Surprise = High, Neutral = Mid, Anger/Sadness/Fear = Low
    mood_map = {
        'joy': 10, 'surprise': 8,
        'neutral': 6,
        'sadness': 4, 'fear': 3, 'anger': 2, 'disgust': 1
    }
    df['Mood_Score'] = df['Emotion'].map(mood_map)

    print("✅ Analysis complete.")
    return df

# --- STEP 5: TASK OPTIMIZATION LOGIC ---
def recommend_tasks(emotion):
    """
    Logic to recommend tasks based on detected mood.
    Refers to PDF Feature 2: Task Recommendation.
    """
    if emotion in ['joy', 'surprise']:
        return "Creative: Brainstorming, Client Pitch, Innovation"
    elif emotion == 'neutral':
        return "Routine: Documentation, Code Review, Reporting"
    elif emotion in ['sadness', 'fear', 'anger']:
        return "Low Stress: Data Cleaning, Learning, Admin Tasks"
    return "General: Daily Sync"

# --- STEP 6: VISUALIZATION DASHBOARD ---
def create_hr_dashboard(df):
    """
    Creates the HR dashboard.
    Refers to PDF Feature 4 & 5: Stress Alerts & Team Analytics.
    """
    print("\n🎨 Generating HR Dashboard...")

    # Calculate Team Mood Trend
    daily_mood = df.groupby('Date')['Mood_Score'].mean().reset_index()

    # Identify Burnout Risk (Employees with low average mood)
    emp_stats = df.groupby('Employee')['Mood_Score'].mean().sort_values().reset_index()

    fig = plotly.subplots.make_subplots(
        rows=2, cols=2,
        subplot_titles=("Team Mood Trend (Last 30 Days)", "Burnout Risk Analysis",
                        "Emotion Distribution", "Recent Task Recommendations"),
        specs=[[{"type": "scatter"}, {"type": "bar"}],
               [{"type": "pie"}, {"type": "table"}]]
    )

    # 1. Team Mood Trend
    fig.add_trace(go.Scatter(x=daily_mood['Date'], y=daily_mood['Mood_Score'],
                             mode='lines+markers', name='Team Morale',
                             line=dict(color='rgb(99, 102, 241)', width=3)), row=1, col=1)

    # 2. Burnout Risk (Bar Chart)
    colors = ['red' if x < 5 else 'green' for x in emp_stats['Mood_Score']]
    fig.add_trace(go.Bar(x=emp_stats['Employee'], y=emp_stats['Mood_Score'],
                         name='Avg Mood Score', marker_color=colors), row=1, col=2)

    # 3. Emotion Distribution (Pie Chart)
    emotion_counts = df['Emotion'].value_counts()
    fig.add_trace(go.Pie(labels=emotion_counts.index, values=emotion_counts.values,
                         name='Emotions', hole=0.4), row=2, col=1)

    # 4. Task Recommendations Table (Latest Entries)
    latest_logs = df.sort_values('Date', ascending=False).head(5)
    latest_logs['Recommended_Task'] = latest_logs['Emotion'].apply(recommend_tasks)

    fig.add_trace(go.Table(
        header=dict(values=['Employee', 'Detected Emotion', 'Recommended Task'],
                    fill_color='paleturquoise', align='left'),
        cells=dict(values=[latest_logs['Employee'], latest_logs['Emotion'], latest_logs['Recommended_Task']],
                   fill_color='lavender', align='left')
    ), row=2, col=2)

    fig.update_layout(
        title="<b>Amdox AI-Powered Task Optimizer Dashboard</b><br><sup>HR Analytics for Employee Well-being</sup>",
        template="plotly_dark",
        height=800,
        showlegend=False
    )

    fig.show()
    return emp_stats

# --- STEP 7: EXECUTION ---
if __name__ == "__main__":
    try:
        # 1. Simulate Data
        df = generate_employee_data()

        # 2. Analyze with AI
        df = analyze_emotions(df)

        # 3. Show Dashboard
        risk_stats = create_hr_dashboard(df)

        # 4. Generate HR Report
        print("\n" + "="*50)
        print("📋 HR EXECUTIVE SUMMARY")
        print("="*50)

        # Identify At-Risk Employees
        at_risk = risk_stats[risk_stats['Mood_Score'] < 5]

        if not at_risk.empty:
            print("⚠️ ALERT: The following employees are showing signs of stress/burnout:")
            for _, row in at_risk.iterrows():
                print(f"   - {row['Employee']} (Mood Score: {row['Mood_Score']:.1f}/10)")
            print("   Action: Schedule 1:1 check-in or adjust workload.")
        else:
            print("✅ Team morale appears healthy. No immediate burnout risks detected.")

        print("-" * 50)
        print("Project demonstrates:")
        print("   - NLP for Emotion Detection (DistilRoBERTa)")
        print("   - Intelligent Task Allocation Logic")
        print("   - People Analytics & Visualization")
        print("="*50)

    except Exception as e:
        print(f"\n❌ Error: {e}")

⏳ Installing AI & Visualization libraries... (This may take a minute)

📝 Simulating 30 days of communication logs for 5 employees...
🧠 Loading NLP Emotion Model (DistilRoBERTa)...


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cpu


⚡ Analyzing employee logs...
✅ Analysis complete.

🎨 Generating HR Dashboard...

❌ Error: module 'plotly' has no attribute 'subplots'
